In [1]:
from music21 import *
%load_ext music21.ipython21

In [2]:
#imports
import schillinger.rhythm as SR
import numpy as np
import itertools
import random

In [3]:
# simple notes show

def showNotes(note_sequece, time_meter, duration_divider):
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    s = stream.Score(id='mainScore')
    p0 = stream.Part(id='part0')
    p0.timeSignature = meter.TimeSignature(TS)
    for i, notes in enumerate(note_sequece):
        notes.quarterLength = notes.quarterLength*duration_divider
        p0.append(notes)
    s.insert(0, p0)
    return s

In [4]:
## PrimaryAxis ## the primary axises can change but this class describes 1 PA for now
#### so 0 can vary!
# what kind of data structure should i use?? scale?

# primary axis PA
# motions in PA = 0,A,B,C and combinations of them
# AXES:
# 0 = no motion = absolute balance
# A = Accending from PA = unbalancing (leading away from 0)
# B = Descending toward PA = balancing (leading to 0)
# C = Accending towards PA = balancing (leading to 0)
# D = Descending from PA = unbalancing (leading away from 0)
# VARIABLES:
# two variables = P for pitch and T for time
# f.e.: ATP
# f.e.2: A4T3P

# generate PA class
# 4 args scale, motion sequence, P, T
# take PAaxes combination f.e. [2xA 1xB 3xD 1xC] or [AABDDDC]
# take variables f.e. P = [2,1,1,2]
# P: f.e. P = [2,1,1,2]
# T: f.e. T = [3,1,2,2,2,3] or so
# combine PAaxes with P and T
# return PA melody

class PrimaryAxis:
    
    def __init__(self, harmony_length, motion_sequence, T, P, meter_nominator):
        
        self.sequence_list = [] # not needed in return
        self.durations = [] # return
        self.pitches = [] # return
        self.seq_length = 0 # return
        
        self.harmony_length = len(motion_sequence)## !
        self.motion_sequence = motion_sequence
        self.P = P
        self.T = T
        
        self.combinePA()
        self.calculate_sequence()
        self.seq_length = len(self.durations)
    
    def combinePA(self):
        
        for i in range(self.harmony_length):
            sequence = self.motion_sequence[i],self.T[i%len(self.T)],self.P[i%len(self.P)]
            self.sequence_list.append(sequence)
        return
    
    def calculate_sequence(self):
        for i, e in enumerate(self.sequence_list):
            if e[0] == "A":
                self.durations.append(e[1])
                self.pitches.append([0, 0 + e[2]])
            elif e[0] == "B":
                self.durations.append(e[1])
                self.pitches.append([0 + e[2], 0])
            elif e[0] == "C":
                self.durations.append(e[1])
                self.pitches.append([0 - e[2],0])
            elif e[0] == "D":
                self.durations.append(e[1])
                self.pitches.append([0, 0 - e[2]])
            elif e[0] == "0":
                self.durations.append(e[1])
                self.pitches.append([0, 0])

def convert_to_music21_sequence(sc, pitches, durations):
    note_seq = []
    tonic = sc.getTonic()
    for i, o in enumerate(pitches):
        for e, p in enumerate(o):
            #c = sc.pitchFromDegree(p+1)
            if p != 0:
                c = sc.next(tonic, direction=p)
            else:
                c = tonic
                
            nota = note.Note(c.nameWithOctave)
            if p >= 0:
                nota.octave = 4
            else:
                nota.octave = 3
            if e is 0:
                nota.duration.quarterLength = durations[i]
            else:
                nota.duration.quarterLength = 1
            note_seq.append(nota)
    return note_seq



In [13]:
# motion_sequence = "AABDDDC"
myscale = scale.HarmonicMinorScale('C4') # not needed for now

motion_sequence = "AB"
time_seqence = [4,4]
pitch_sequence = [5,5]
harmony_length = 8 #not needed for now
meter_nominator = 4
PA = PrimaryAxis(harmony_length, motion_sequence, time_seqence, pitch_sequence, meter_nominator)

hm = scale.HarmonicMinorScale("C4")
#myscale = scale.ConcreteScale(pitches = ["C4", "D4", "E-4", "F4", "G4", "A-4", "B-4"]) 
myscale = hm
pitches = convert_to_music21_sequence(myscale,PA.pitches, PA.durations)
scales = [[myscale.pitchFromDegree(x+1).name for x in range(7)]]

melody = [[p.name for p in pitches]]
scales = [[myscale.pitchFromDegree(x+1).name for x in range(7)]]
seq_rhythm = [p.duration.quarterLength for p in pitches] #for own melody test


In [14]:
def convert_to_music21_sequence(sc, pitches, durations):
    note_seq = []
    tonic = sc.getTonic()
    for i, o in enumerate(pitches):
        for e, p in enumerate(o):
            #c = sc.pitchFromDegree(p+1)
            if p != 0:
                c = sc.next(tonic, direction=p)
            else:
                c = tonic
                
            nota = note.Note(c.nameWithOctave)
            if p >= 0:
                nota.octave = 4
            else:
                nota.octave = 3
            if e is 0:
                nota.duration.quarterLength = durations[i]
            else:
                nota.duration.quarterLength = 1
            note_seq.append(nota)
    return note_seq

In [15]:
pitches = convert_to_music21_sequence(myscale,PA.pitches, PA.durations)

In [16]:
s = showNotes(pitches,[4,4],2)
s.show('musicxml')

<music21.scale.HarmonicMinorScale C harmonic minor>


In [9]:
[myscale.pitchFromDegree(x+1).name for x in range(7)]

['C', 'D', 'E-', 'F', 'G', 'A-', 'B']

In [10]:
#sp = midi.realtime.StreamPlayer(s)
#sp.play()
#s.show('midi')
#s.show('musicxml')
#s.show()
#scales

In [8]:
# plotting

import matplotlib.pyplot as plt


def show_graph(common_product):
    
    length = len(common_product)
    x_vals = [x for x in range(length)]
    #fig, ax = plt.subplots(figsize=(20, 10))
    fig, ax = plt.subplots()
    ax.set_xlim(xmin=-1, xmax=length)
    ax.set_ylim(ymin=-12,ymax=12)
    
    ax.step(x_vals, common_product, color='black')
    
    ax.grid()
    #ax.set_xticks([x+x for x in convert_to_plot(resultant)])
    #ax.set_xticklabels([])
    #ax.set_yticklabels([])
    ax.set_xticks(x_vals)
    ax.set_yticks([x for x in range(-12,12)])
    ax.grid(True)
    plt.show();
        

In [21]:
# oscillator 
def oscillator(osc_seq,osc_type, amount, sin=True):
    
    def oscillate(l,h):
        #if sin:
        if random.choice([0,1]):
            osc.append(list(np.linspace(l, -h, 2)))
        else:
            osc.append(list(np.linspace(-l, h, 2)))
    osc = []
    '''a,b,c,d--- cb,cb,ad,da--parallel'''#
    
    for i in range(amount):
        e = osc_seq[i%len(osc_seq)]
        if e != 0:
            if osc_type == 'a':
                oscillate(0,e)
            if osc_type == 'd':
                oscillate(e,0)
            if osc_type == 'da':
                oscillate(e,e)
        
    return_value = [list(itertools.chain(*osc))][0]
    return return_value


In [32]:
#
melo = []
for b in PA.pitches:
    a = []
    if b[0]>b[1]:
        #print([x for x in range(b[0],b[1]-1,-1)])
        a = [x for x in range(b[0],b[1]-1,-1)]
    else:
        a = [x for x in range(b[0],b[1]+1)]
        #print([x for x in range(b[0],b[1]+1)])
    nums = list(np.linspace(a[0], a[-1], 5))
    melodic_oscillation = oscillator([round(x) for x in nums], random.choice(["a","d","da","da"]), 4 ,sin=True)
    #show_graph(melodic_oscillation)
    melo.append(melodic_oscillation)
    print(a, melodic_oscillation, [round(x) for x in nums])
#start_end = 0
#cuvre = oscillator([3,2,1], start_end, "da", sin=True)
#show_graph(cuvre)
melo = [list(itertools.chain(*list(melo)))]

[0, 1, 2, 3, 4, 5] [0.0, 1.0, 0.0, 2.0, 0.0, -4.0] [0.0, 1.0, 2.0, 4.0, 5.0]
[5, 4, 3, 2, 1, 0] [5.0, -5.0, -4.0, 4.0, 2.0, -2.0, 1.0, -1.0] [5.0, 4.0, 2.0, 1.0, 0.0]


In [18]:
def convert_to_music21_simle_sequence(sc, pitches, durations):
    note_seq = []
    tonic = sc.getTonic()
    
    for i, o in enumerate(pitches):
        for e, p in enumerate(o):
            #c = sc.pitchFromDegree(p+1)
            if p != 0:
                c = sc.next(tonic, direction=int(p))
            else:
                c = tonic
                
            nota = note.Note(c.nameWithOctave)
            if p >= 0:
                nota.octave = 4
            else:
                nota.octave = 3
            nota.duration.quarterLength = durations[e%len(durations)]
            note_seq.append(nota)
    return note_seq

In [19]:
durations=[2,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1]
ns = convert_to_music21_simle_sequence(myscale, melo, durations)

In [20]:
s = showNotes(ns,[4,4],.5)
s.show('musicxml')

In [ ]:
def SecondaryAxis(time_units,oscillation_types,PA):
    global_length = sum(PA.duration)
    for i in range(global_length):
        #oscillate a cycle
        
    

PA is a graph or guide of motion, it has a direction in time and has a value !
the SA has motion in time and uses the PA for synchronization (synchronizes with PA)

PA affect the melody indirectly thru the SA and NOT directly for itself!
SA affects the melody directly and uses the PA as guide for !global direction.

so SA needs:
       -oscillation or motion (types = a b c d ad bc and parallel)
       -inscribed (sin)(bellow PA for a b and above PA for c d ) or  or ascribed(cos)
       -time units
       -global length m.b. = PA length!
       -and PA as guide
       
PA length and SA length should synchronize.

the time units of PA and SA are independent
the pitch units of SA are dependent on oscillation and PA as a guide!

pseudo code:
time_units = 2+2+2+2+1+1+1+1+1+1+1 or 3+1+2+2+1+3 or something
oscillation_type = 2a+b+c+2d
PA = a5T5P+b4T5P (T = time, P = pitch)

SA(time_units, oscillation_type , PA):
    length = len(PA)
    for l in range()
    

In [ ]:
# oscillator 
def oscillator(osc_seq,osc_type, amount, sin=True):
    osc = []
    
    def sinus():
        return
    def cosinus():
        return
    def sincos():
        return
        
    return_value = [list(itertools.chain(*osc))][0]
    return return_value

In [106]:
np.sin(np.deg2rad(0))

0.0

In [241]:
cf = alpha.counterpoint.species.getRandomCF(mode="C")
sorted(list(cf.keys()))
cf['mode']

Exception: Cantus firmus selection by mode does not yet exist

In [237]:
#counterpoint music21 TAKES FOREVER!!!
cf = alpha.counterpoint.species.getRandomCF()
c = converter.parse('tinynotation: 4/4 '+cf['notes'])
bass = stream.Stream(c)
bass = bass.flat

sc1 = scale.MinorScale("D4")
cpp = alpha.counterpoint.species.ModalCounterpoint(bass, bass)
fs = cpp.generateFirstSpecies(bass,sc1)

s = stream.Score(id='mainScore')
p0 = stream.Part(id='melody')
p0.timeSignature = meter.TimeSignature("4/4")
p1 = stream.Part(id='chords')
p1.timeSignature = meter.TimeSignature("4/4")
p0.append(fs)
p1.append(bass)
p1 = p1.transpose(-12)
s.insert(0, p0)
s.insert(0, p1)
s.show('musicxml')

KeyboardInterrupt: 

In [233]:

#cpp.getValidSecondVoice(stream1=cf, minorScale=sc1, choice='random')

In [236]:
s = stream.Score(id='mainScore')
p0 = stream.Part(id='melody')
p0.timeSignature = meter.TimeSignature("4/4")
p1 = stream.Part(id='chords')
p1.timeSignature = meter.TimeSignature("4/4")
p0.append(fs)
p1.append(bass)
p1 = p1.transpose(-12)
s.insert(0, p0)
s.insert(0, p1)
s.show('musicxml')